## Préparation du jeu de données pour le modèle DIST_REL_C_RNN

Nous allons dans ce notebook préparer un nouveau jeu de données pour le nouveau modèle DIST_REL_C_RNN.
Nous n'allons pas le créer à partir de toutes les données que nous avons car le traitement risque d'être trop long, sachant que l'algorithme de création est constitué de boucles imbriquées.

Pour chaque couple d'atomes de carbone (c1, c2) partageant une liaison, notre jeu de données sera constitué de la façon suivante :

* Entrée RN : pour chaque atome de la molécule (hors c1, c2):
    * Numéro atomique (one-hot encoding)
    * Masse atomique
    * Distances avec c1 et c2
    * Classe positionnelle de l'atome (one-hot encoding). Pour la déterminer, on compare la  position de l'atome par rapport aux deux plans P1 (resp. P2) de vecteur normal C1C2 et passant par point C1 (resp. C2). Si l'atome est entre les deux plans il aura pour classe 'C', et s'il est derrière le plan P1 (resp. P2) il aura pour classe 'G' (resp 'D').
    

* Cible RN : Distance entre c1 et c2

Note : Pour limiter la taille de l'entrée du RN (one-hot encoding du numéro atomique), on s'intéresse seulement au molécules ne contenant pas d'atomes plus lourds que le fluor.

#### Chemin des fichiers

In [3]:
train_riken_reduced_loc = "../data/train_set_riken_v2_reduced.h5"
test_riken_reduced_loc = "../data/test_set_riken_v2_reduced.h5"

train_prepared_input_loc = "../data/DIST_REL_C_RNN/train_set_prepared_input(+ids).h5"
train_labels_loc = "../data/DIST_REL_C_RNN/train_set_labels(+ids).h5"

test_prepared_input_loc = "../data/DIST_REL_C_RNN/test_set_prepared_input(+ids).h5"
test_labels_loc = "../data/DIST_REL_C_RNN/test_set_labels(+ids).h5"

minimal_reduced_loc = "../data/minimal_set_riken_v2_reduced.h5"
minimal_prepared_input_loc = "../data/DIST_REL_C_RNN/minimal_prepared_input(+ids).h5"
minimal_labels_loc = "../data/DIST_REL_C_RNN/minimal_labels.h5(+ids)"

#### Calcul de la distance entre deux atomes

In [1]:
import numpy as np

def calcul_distance(pt1, pt2):
    """ Renvoie la distance entre deux points représentés par leurs coordonnées (x, y, z) dans deux tableaux
    de forme (1, 3)"""
    return np.sqrt(np.sum(np.square(np.diff(np.array([pt1, pt2]), axis=0))))

#### Fonction de test si les deux atomes de carbone partagent une liaison

Selon les mesures moyennes des distances carbone-carbone (http://hydra.vcp.monash.edu.au/modules/mod2/bondlen.html), on considère ici que deux atomes de carbone partagent une liaison si leur distance est comprise entre 1.15 Å et 1.60 Å.

In [241]:
def existe_liaison_C(coords_c1, coords_c2):
    dist = calcul_distance(coords_c1, coords_c2)
    return dist >= 1.15 and dist <= 1.6

#### Fonction renvoyant les distances d'un atome à deux autres atomes

In [2]:
def calcul_distances(pt1, pt_ref1, pt_ref2):
    return np.array([calcul_distance(pt1, pt_ref1), calcul_distance(pt1, pt_ref2)])

#### Fonction renvoyant la classe positionnelle d'un atome en fonction de sa position et des positions des deux atomes de référence. Renvoie le résultat sous forme de one-hot-encoding

In [3]:
def get_classe_pos(pt, c1, c2):
    
    # Initialisation du tableau de sortie
    classes_pos = np.zeros(shape=(3,))
    
    # Calcul du vecteur C1_C2
    vect = np.diff([c1, c2], axis=0)
    
    # Déclaration de la matrice contenant les coordonnées des trois points g, c et d
    gcd_pos = np.empty(shape=(3, 3))
    
    # Calcul du point c
    gcd_pos[1] = np.divide([np.sum([c1, c2], axis=0)], 2)

    # Calcul des points g et d
    gcd_pos[0] = gcd_pos[1] - vect
    gcd_pos[2] = gcd_pos[1] + vect
        
    # Calcul de la matrice des positions répétées du point
    pos = np.tile(pt, 3).reshape(3, 3)
    
    # Calcul des distances du point avec les points g, c et d
    dists = np.sqrt(np.sum(np.square(np.diff([gcd_pos, pos], axis=0)[0]), axis=1))
    
    # On renvoie le résultat encodé avec la méthode du one hot encoding
    classes_pos[np.argmin(dists)] = 1
    return classes_pos
    

#### Fonction renvoyant le numéro de l'atome encodé en one-hot encoding (pour les atomes des deux premières lignes du tableau périodique)

In [4]:
def get_anum_one_hot(z):
    
    if not 0 < z < 10:
        raise RuntimeError("Atomic number must be between 1 and 9")
    
    one_hot = np.zeros(shape=(9,))
    one_hot[int(z)-1] = 1
    return one_hot


#### Fonction renvoyant les données préparées pour le RN pour une molécule

In [5]:
def donnees_prep_RN_mol(coords_mol, anums_mol, amasses_mol):
    
    taille_mol = len(coords_mol)
    
    inputs_RN = []
    targets_RN = []
    
    # On itère sur tous les atomes de la molécule
    for i in range(taille_mol):
        
        # On ne s'intéresse qu'aux atomes de carbone
        if anums_mol[i] == 6.:
            
            # On parcourt tous les atomes suivants
            for j in range(i+1, taille_mol):
                
                # On ne s'intéresse qu'aux couples d'atomes de carbone partageant une liaison
                if anums_mol[j] == 6 and existe_liaison_C(coords_mol[i], coords_mol[j]):
                    
                    # On calcule la distance entre les deux atomes du couple
                    dist_c_c = calcul_distance(coords_mol[i], coords_mol[j])
                    
                    # On initialise l'entrée du RN pour le couple courant
                    input_rn = np.empty(shape=(taille_mol-2, 15))
                    
                    input_rn_idx = 0
                    
                    # On itère sur tous les atomes de la molécule (en dehors des deux atomes de carbone)
                    for k in range(taille_mol):
                        if k != i and k != j:
                            
                            # On enregistre le numéro et la masse atomique de la molécule
                            input_rn[input_rn_idx][:9] = get_anum_one_hot(anums_mol[k])
                            input_rn[input_rn_idx][9] = amasses_mol[k]
                            
                            # On enregistre les distances de l'atome aux deux atomes de carbone
                            input_rn[input_rn_idx][10:12] = calcul_distances(coords_mol[k], coords_mol[i],
                                                                         coords_mol[j])
                            
                            # On enregistre la classe positionnelle de l'atome par rapport aux deux atomes
                            # de carbone
                            input_rn[input_rn_idx][12:15] = get_classe_pos(coords_mol[k], coords_mol[i], 
                                                                        coords_mol[j])    

                            input_rn_idx += 1
                            
                    # On aplatit l'entrée du RN
                    input_rn = np.array(input_rn)
                    input_rn = input_rn.reshape(-1,)
                            
                    # On enregistre l'entrée et la cible du RN pour l'atome courant
                    inputs_RN.append(input_rn)
                    
                    # On enregistre la distance cible pour le couple d'atomes courant (en mÅ)
                    targets_RN.append(dist_c_c*1000)   
                    
        else:
            # Les carbones étant les premiers atomes de la molécule dans les données, on arrête la boucle
            # pour gagner du temps d'exécution
            break
                
    return np.array(inputs_RN), np.array(targets_RN)
            
        


#### Définition de la fonction de création du jeu de données

In [1]:
import h5py
import time

def generer_donnees(original_dataset_loc, prepared_input_loc, labels_loc, nb_mol, batch_size):
    """ Génère les entrées préparées et les labels pour le jeu courant, à partir des nb\_mol
    premières molécules du jeu """
    
    start_time = time.time()    
    
    # On charge les données du jeu original en mémoire
    original_dataset_h5 = h5py.File(original_dataset_loc, "r")
    input_coords = np.array(original_dataset_h5["riken_coords"][:nb_mol])
    input_masses = np.array(original_dataset_h5["amasses"][:nb_mol])
    input_nums = np.array(original_dataset_h5["anums"][:nb_mol])
    input_pubchem_ids = np.array(original_dataset_h5["pubchem_id"][:nb_mol])
    
    # On créé les jeux de données d'entrée du RN et de labels
    input_rn_dataset_h5 = h5py.File(prepared_input_loc, 'w')
    labels_dataset_h5 = h5py.File(labels_loc, 'w')    
    
    try:
        varlen_floatarray = h5py.special_dtype(vlen=np.dtype("float32"))
        
        input_dataset = input_rn_dataset_h5.create_dataset("inputs", maxshape=(None,), shape=(0,),
                                           dtype=varlen_floatarray, compression="gzip", 
                                           chunks=True)
        
        ids_dataset = input_rn_dataset_h5.create_dataset("pubchem_ids", maxshape=(None,), shape=(0,),
                                          dtype=np.float32, compression="gzip",
                                          chunks=True)
                                            

        target_dataset = labels_dataset_h5.create_dataset("targets", maxshape=(None,), shape=(0,),
                                           dtype=np.float32, compression="gzip", 
                                           chunks=True)


        datasets_curr_idx = 0
        
        # On créé les tableaux pour le premier batch de traitement
        inputs_batch = []
        outputs_batch = []
        ids_batch = []

        # On parcourt toutes les molécules du jeu original
        for ori_idx in range(nb_mol):
            
            
            # On ne s'intéresse pas aux molécules contenenant des atomes plus lourds que le fluor (contrainte
            # imposée par l'utilisation d'un one-hot encoding pour les numéros atomiques)
            if max(input_nums[ori_idx])<10:
                
                # On calcule les entrées et sortie du RN pour la molécule courante (il peut y avoir plusieurs
                # entrées et plusieurs sorties s'il existe plusieurs liaisons C-C dans la molécule)
                curr_inputs_np, curr_outputs_np, curr_ids_np = donnees_prep_RN_mol(input_coords[ori_idx].reshape(-1, 3), 
                                                                    input_nums[ori_idx],
                                                                    input_masses[ori_idx],
                                                                    input_pubchem_ids[ori_idx])
                
                # On ajoute les entrées et sorties aux tableaux du batch courant
                inputs_batch.extend(curr_inputs_np)
                outputs_batch.extend(curr_outputs_np)
                ids_batch.extend(curr_ids_np)
                   
            # On écrit les tableaux h5 du batch courant
            if ori_idx % batch_size == 0 or ori_idx == len(input_nums)-1:
                
                print("Molécule courante : "+str(ori_idx))
    
                # On redimensionne les datasets
                input_dataset.resize((input_dataset.shape[0]+len(inputs_batch),))
                target_dataset.resize((target_dataset.shape[0]+len(inputs_batch),))
                ids_dataset.resize((ids_dataset.shape[0]+len(inputs_batch),))
                
                # On écrit les données dans les datasets
                input_dataset[datasets_curr_idx:datasets_curr_idx+len(inputs_batch)] = np.array(inputs_batch)
                target_dataset[datasets_curr_idx:datasets_curr_idx+len(inputs_batch)] = np.array(outputs_batch)
                ids_dataset[datasets_curr_idx:datasets_curr_idx+len(inputs_batch)] = np.array(ids_batch)
                
                datasets_curr_idx += len(inputs_batch)
                
                # On réinitialise les tableaux
                inputs_batch = []
                outputs_batch = []
                ids_batch = []
                
        input_rn_dataset_h5.flush()
        labels_dataset_h5.flush()
        
        print(str(len(input_dataset))+" exemples créés")
        print("--- %s seconds ---" % (time.time() - start_time))
                
    
    finally:
        input_rn_dataset_h5.close()
        labels_dataset_h5.close()
        original_dataset_h5.close()

    
    

#### Génération des exemples pour le jeu minimal

In [247]:
print("Minimal set")
generer_donnees(minimal_reduced_loc, minimal_prepared_input_loc, minimal_labels_loc, 1000, 1000)

Minimal set
Molécule courante : 0
Molécule courante : 999
5635 exemples créés
--- 14.161330223083496 seconds ---


#### Génération des exemples pour le jeu d'entraînement

In [248]:
print("Train set")
generer_donnees(train_riken_reduced_loc, train_prepared_input_loc, train_labels_loc, 400000, 10000)

Train set


ValueError: max() arg is an empty sequence

##### Sortie

```
2214225 exemples créés
--- 11557.782796144485 seconds ---
```

#### Génération des exemples pour le jeu de test

In [ ]:
print("Test set")
generer_donnees(test_riken_reduced_loc, test_prepared_input_loc, test_labels_loc, 80000, 5000)

##### Sortie
```
442853 exemples créés
--- 2428.7579131126404 seconds ---
```